# Jupyter Notebook for Project "Comparison of LLM Prompting Techniques"

In [1]:
import pandas as pd
import mlflow
import mlflow.pyfunc
import sacrebleu
from llama_cpp import Llama
import time


## 1 Data Loading
In the first step we import the given translations as pandas Dataframes and print a quick overview of the dataframe.

In [2]:
data = pd.read_pickle('machine_translation.pkl')
data

,complexity,text_german,text_english
0,easy,Felix hat es satt: Ständig ist Mama unterwegs....,Felix is fed up: Mom is always on the go. But ...
1,news_gen,Die rund 1.400 eingesetzten Beamten haben demn...,"The approximately 1,400 deployed officers have..."
2,news_spec,"Der Staatschef hat zugleich aber das Recht, vo...",The head of state also has the right to appoin...
3,pop_science,Dass der Klimawandel die Hitzewellen in Südasi...,There is no question that climate change is in...
4,science,"Der DSA-110, der sich am Owens Valley Radio Ob...","The DSA-110, situated at the Owens Valley Radi..."


In [3]:
data_info = pd.DataFrame()
data_info['complexity'] = data['complexity']
data_info['text_german_length'] = data['text_german'].str.len()
data_info['text_english_length'] = data['text_english'].str.len()
data_info

,complexity,text_german_length,text_english_length
0,easy,485,415
1,news_gen,296,280
2,news_spec,518,484
3,pop_science,542,521
4,science,1003,827


In [4]:
from enum import Enum


class Language(Enum):
    ENGLISH = 'English'
    GERMAN = 'German'


class Complexity(Enum):
    EASY = 'easy'
    NEWS_GEN = 'news_gen'
    NEWS_SPEC = 'news_spec'
    POP_SCIENCE = 'pop_science'
    SCIENCE = 'science'


ALL_COMPLEXITIES = list(Complexity)

***
## 2 Model Loading
In the second step we import the AI-Models which are given in the specified task. For doing so we use the `llama-cpp-python` library (further documentation can be found [here](https://github.com/abetlen/llama-cpp-python)) and import the models directly from [huggingface](https://huggingface.co/).

Quick overview and installation guide of llama.cpp:
- https://www.datacamp.com/tutorial/llama-cpp-tutorial
- https://christophergs.com/blog/running-open-source-llms-in-python

In [5]:
# Configuration of the models
MODELS = {
    'gemma': {
        'repo_id': 'lmstudio-ai/gemma-2b-it-GGUF',
        'filename': 'gemma-2b-it-q8_0.gguf',
    },
    'llama32': {
        'repo_id': 'hugging-quants/Llama-3.2-3B-Instruct-Q8_0-GGUF',
        'filename': 'llama-3.2-3b-instruct-q8_0.gguf',
    },
    'llama31': {
        'repo_id': 'lmstudio-community/Meta-Llama-3.1-8B-Instruct-GGUF',
        'filename': 'Meta-Llama-3.1-8B-Instruct-Q5_K_M.gguf',
    },
    #'aya23': {
    #    'repo_id': 'bartowski/aya-23-35B-GGUF',
    #    'filename': 'aya-23-35B-Q5_K_M.gguf',
    #},
}

In [6]:
def create_llama_model(repo_id, filename, n_ctx=None):
    try:
        if n_ctx is None:
            # default of llama_cpp
            n_ctx = 512
        if repo_id is not None and filename is not None:
            model = Llama.from_pretrained(
                repo_id=repo_id,
                filename=filename,
                n_ctx=n_ctx,
                # n_gpu_layers=n_gpu_layers,
                #n_threads=8,
                verbose=False,
            )
            print(f"Model {repo_id} erfolgreich geladen mit n_ctx={n_ctx}")
            return model
        else:
            return None
    except Exception as e:
        print(f"Fehler beim Laden von {filename}: {e}")
        return None

***

## 3 Pipeline

### 3.1 Model Interaction

In [7]:
def translate(model, prompt, reference_translation):
    # we estimate the needed max_tokens based on the tokenized prompt and reference_translation
    token_length_ref = len(model.tokenize(reference_translation.encode('utf-8')))
    token_length_prompt = len(model.tokenize(prompt.encode('utf-8')))
    # the model should not need more tokens than this
    estimated_max_tokens = (token_length_prompt + token_length_ref) * 1.5

    response = model(prompt, max_tokens=estimated_max_tokens, echo=False)
    # print(response['choices'][0]['text'])
    return response['choices'][0]['text']

### 3.2 Metrics Calculation
[GitHub Repo to MetricX](https://github.com/google-research/metricx)

In [8]:
import subprocess
import json
import os


def calculate_metricx_score(source, reference, hypothesis):
    '''
    Calculates the MetricX-score based on source, reference, and hypothesis using metricx24.
    We are currently using the metricx-24-hybrid-large-v2p6-bfloat16 model but there are also other options
        as can be seen here: https://github.com/google-research/metricx

    Args:
        source: The source text (String).
        reference: The reference translation (String).
        hypothesis: The hypothesis translation (String).

    Returns:
        The calculated score as a float or None in case of an error.
    '''

    data = [{'id': '1', 'source': source, 'reference': reference, 'hypothesis': hypothesis}]

    # Create temporary JSONL files
    input_file = './temp_input.jsonl'
    output_file = './temp_output.jsonl'
    model = 'google/metricx-24-hybrid-large-v2p6-bfloat16'

    try:
        with open(input_file, 'w', encoding='utf-8') as f:
            for entry in data:
                json.dump(entry, f)
                f.write('\n')

        command = [
            'python', '-m', 'metricx24.predict',
            '--tokenizer', 'google/mt5-xl',
            '--model_name_or_path', model,
            '--max_input_length', '1536',
            '--batch_size', '1',
            '--input_file', input_file,
            '--output_file', output_file
        ]

        process = subprocess.Popen(
            command,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            text=True,
            bufsize=1,
            universal_newlines=True
        )

        # Capture output and errors (optional, can be useful for debugging)
        #for line in process.stdout:
        #    print(line, end='')
        #for line in process.stderr:
        #    print(f'ERROR: {line}', end='')

        process.wait()

        if process.returncode != 0:
            print(f'Error executing metricx24. Return code: {process.returncode}')
            return None

        # Read score from the output file
        with open(output_file, 'r', encoding='utf-8') as f:
            for line in f:
                try:
                    output_data = json.loads(line)
                    score = float(output_data.get('prediction'))  # Ensure that 'score' exists
                    return score
                except (json.JSONDecodeError, ValueError, AttributeError):
                    print('Error parsing the output file.')
                    return None

        return None  # If no valid line was found in the output file

    finally:
        # Remove temporary files
        try:
            os.remove(input_file)
            os.remove(output_file)
        except FileNotFoundError:
            pass  #If the files don't exist for some reason, the error is caught




In [9]:
from rouge_score import rouge_scorer


def evaluate_translation(source, reference, hypothesis):
    # Note that BLEU and chrF Scores can only be between 0 and 100
    #    but sacreblue returns floats as percentage values
    # --> so the scores are between 0 and 100)
    bleu_score = sacrebleu.corpus_bleu([hypothesis], [[reference]]).score
    chrf_score = sacrebleu.corpus_chrf([hypothesis], [[reference]]).score

    metricx_score = calculate_metricx_score(source, reference, hypothesis)
    if metricx_score is None:
        metricx_score = -1

    rougel_scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
    rougel_score = rougel_scorer.score(reference, hypothesis)

    return {'BLEU': bleu_score,
            'chrF': chrf_score,
            'rougeL': (rougel_score['rougeL'].fmeasure * 100),
            'MetricX': metricx_score}


In [10]:
#evaluate_translation("Felix hat es satt: Ständig ist Mama unterwegs. Doch warum das so ist, will ihm niemand verraten. Für Felix ist daher klar: Seine Mutter ist eine Geheimagentin. Als er an seinem zehnten Geburtstag einen rätselhaften Brief erhält, scheint sich seine Vermutung zu bestätigen. Zusammen mit seiner besten Freundin Lina macht er sich daran, das Geheimnis um Mamas Arbeit zu lüften. Ehe sie sich versehen, stecken die beiden mitten in ihrem ersten spannenden Fall als angehende Geheimagenten.", "Felix is fed up: Mom is always on the go. But nobody will tell him why that is. For Felix, it's clear: his mother is a secret agent. When he receives a mysterious letter on his tenth birthday, his suspicion seems to be confirmed. Together with his best friend Lina, he sets out to uncover the secret of mom's job. Before they know it, the two are in the middle of their first exciting case as budding secret agents.", "\n\n**English Translation:**\n\nFelix had sat: Mama was constantly on the go. But why this is the case, no one will tell him. Therefore, clear to Felix: his mother is a covert agent. When he receives a cryptic letter on his eleventh birthday, it seems his suspicion is confirmed. Together with his best friend Lina, he starts unraveling the mystery of his mother's job. When they finally manage to solve the case, they stick to their first exciting clue like detectives.")

### 3.3 Logging to MLFLow

In [11]:
def log_to_mlflow(experiment_name, template_name, metrics, prompt_type, model_name, complexity, target_language, tmp_result,
                  prompt_language):
    experiment = mlflow.get_experiment_by_name(experiment_name)

    if experiment:
        if experiment.lifecycle_stage == 'deleted':
            mlflow.tracking.MlflowClient().restore_experiment(experiment.experiment_id)
            #mlflow.delete_experiment(experiment.experiment_id)
    else:
        mlflow.create_experiment(experiment_name)

    mlflow.set_experiment(experiment_name)
    with mlflow.start_run(run_name=f'{model_name}/{complexity}/{template_name}'):
        mlflow.log_param('model', model_name)
        mlflow.log_param('complexity', complexity)
        mlflow.log_param('prompt_type', prompt_type)
        mlflow.log_param('target_language', target_language)
        mlflow.log_param('prompt_language', prompt_language)
        for key, value in metrics.items():
            mlflow.log_metric(key, value)

        tmp_result.to_json('tmp_results.json', index=False)
        mlflow.log_artifact('tmp_results.json')
        mlflow.end_run()


### 3.4 Pipeline Composition

In [12]:
import gc


def run_pipeline(texts):
    results = pd.DataFrame(
        columns=['model', 'complexity', 'prompt_type', 'prompt', 'source_text', 'hypothesis', 'reference', 'metrics',
                 'prompt_language'])
    mlflow.set_tracking_uri(uri='http://127.0.0.1:5000')

    for model_name, model_config in MODELS.items():
        for _, row in texts.iterrows():
            model = createModel(model_config, row)
            complexity_enum = next(c for c in Complexity if c.value == row['complexity'])

            # Übersetzung Deutsch -> Englisch
            for template_name, template_data in PROMPT_TEMPLATES_GERMAN_ENGLISH.items():
                if pd.notna(row['text_german']) and complexity_enum in template_data['complexities']:
                    results = execute_mlflow_run(template_name, complexity_enum.value, model, model_name, Language.ENGLISH, results,
                                                 row['text_german'], row['text_english'], template_data)

            # Übersetzung Englisch -> Deutsch
            for template_name, template_data in PROMPT_TEMPLATES_ENGLISH_GERMAN.items():
                if pd.notna(row['text_english']) and complexity_enum in template_data['complexities']:
                    results = execute_mlflow_run(template_name, complexity_enum.value, model, model_name, Language.GERMAN, results,
                                                 row['text_english'], row['text_german'], template_data)

            # we dont need the model anymore so we delete it
            del model
            gc.collect()
    return results


def createModel(model_config, row):
    # at first we just use the dummyModel for tokenization
    dummyModel = create_llama_model(model_config['repo_id'], model_config['filename'])
    combined_text = f"{row['text_german']} {row['text_english']}"
    text_tokens = len(dummyModel.tokenize(combined_text.encode('utf-8')))
    # we want to tokenize the longest template
    max_promp_template = max(
        (t['template'] for d in (PROMPT_TEMPLATES_GERMAN_ENGLISH, PROMPT_TEMPLATES_ENGLISH_GERMAN) for t in d.values()),
        key=len)
    prompt_tokens = len(dummyModel.tokenize(max_promp_template.encode('utf-8')))

    # now we delete the dummyModel and then create the final model based on the estimated_max_tokens
    del dummyModel
    gc.collect()
    estimated_max_tokens = (text_tokens + prompt_tokens) * 1.5
    n_ctx = int(estimated_max_tokens * 1.1)
    print(f"estimated_max_tokens: {estimated_max_tokens}; n_ctx: {n_ctx}")
    model = create_llama_model(model_config['repo_id'], model_config['filename'], n_ctx=n_ctx)
    return model


def execute_mlflow_run(template_name, complexity, model, model_name, target_language: Language, results, source_text,
                       reference_text, template_data):
    prompt = template_data['template'].format(text=source_text)
    prompt_language = template_data['prompt_language']
    prompt_type = template_data['prompt_type']

    start_time_translation = time.time()
    hypothesis = translate(model, prompt, reference_text)
    end_time_translation = time.time()
    print('Prompt finished in (seconds): ', round(end_time_translation - start_time_translation, 2))
    metrics = evaluate_translation(source=source_text, reference=reference_text, hypothesis=hypothesis)
    print('Metric Calculation in (seconds): ', round(time.time() - end_time_translation, 2))

    tmp_result = pd.DataFrame([{
        'model': model_name,
        'complexity': complexity,
        'prompt_type': prompt_type,
        'prompt': prompt,
        'source_text': source_text,
        'hypothesis': hypothesis,
        'reference_text': reference_text,
        'metrics': metrics,
        'prompt_language': prompt_language.value  # Hier .value für den Stringwert
    }])

    # MLflow-Logging
    experiment_name = f'{model_name}_{complexity}'

    log_to_mlflow(experiment_name, template_name, metrics, prompt_type, model_name, complexity, target_language.value, tmp_result,
                  prompt_language.value)

    # Ergebnis speichern
    results = pd.concat([
        results,
        tmp_result
    ], ignore_index=True)
    return results

### 3.5 Prompt Composition


In [13]:
# Beispiel für ausgewählte complexities: 'complexities': [Complexity.EASY, Complexity.NEWS_GEN]

PROMPT_TEMPLATES_ENGLISH_GERMAN = {
    'zero_shot_style_persona_reread_to-de_en_1': {
         'template': 'Imagine you are a professional translator with years of experience. Your specialty is to translate texts from one language into another precisely and while maintaining the original tone, using direct translations. Your task is to translate the following English text into German without asking any questions or providing further explanations: \"{text}\" Now read the instruction again: Imagine you are a professional translator with years of experience. Your specialty is to translate texts from one language into another precisely and while maintaining the original tone, using direct translations. Your task is to translate the English text into German without asking any questions or providing further explanations.',
         'prompt_language': Language.ENGLISH,
         'prompt_type': 'zero_shot_style_persona_reread',
         'complexities': ALL_COMPLEXITIES
    },
    'zero_shot_style_persona_reread_to-de_de_1': {
        'template': 'Stell dir vor, du bist ein professioneller Übersetzer mit jahrelanger Erfahrung. Deine Spezialität ist es, Texte mithilfe von direkten Übersetzungen präzise und unter Beibehaltung des ursprünglichen Tons von einer Sprache in eine andere zu übertragen. Deine Aufgabe ist es, ohne Rückfragen oder weiterführende Erklärungen den folgenden englischen Text ins Deutsche zu übersetzen: \"{text}\" Lies dir die Anweisung jetzt nochmal durch: Stell dir vor, du bist ein professioneller Übersetzer mit jahrelanger Erfahrung. Deine Spezialität ist es, Texte mithilfe von direkten Übersetzungen präzise und unter Beibehaltung des ursprünglichen Tons von einer Sprache in eine andere zu übertragen. Deine Aufgabe ist es, ohne Rückfragen oder weiterführende Erklärungen den englischen Text ins Deutsche zu übersetzen.',
        'prompt_language': Language.GERMAN,
        'prompt_type': 'zero_shot_style_persona_reread',
        'complexities': ALL_COMPLEXITIES
    },
    # ... weitere Einträge
}

PROMPT_TEMPLATES_GERMAN_ENGLISH = {
    'zero_shot_style_persona_reread_to-en_en_1': {
        'template': 'Imagine you are a professional translator with years of experience. Your specialty is to translate texts from one language into another precisely and while maintaining the original tone, using direct translations. Your task is to translate the following German text into English without asking any questions or providing further explanations: \"{text}\" Now read the instruction again: Imagine you are a professional translator with years of experience. Your specialty is to translate texts from one language into another precisely and while maintaining the original tone, using direct translations. Your task is to translate the German text into English without asking any questions or providing further explanations.',
        'prompt_language': Language.ENGLISH,
        'prompt_type': 'zero_shot_style_persona_reread',
         'complexities': ALL_COMPLEXITIES
    },
    'zero_shot_style_persona_reread_to-en_de_1': {
        'template': 'Stell dir vor, du bist ein professioneller Übersetzer mit jahrelanger Erfahrung. Deine Spezialität ist es, Texte mithilfe von direkten Übersetzungen präzise und unter Beibehaltung des ursprünglichen Tons von einer Sprache in eine andere zu übertragen. Deine Aufgabe ist es, ohne Rückfragen oder weiterführende Erklärungen den folgenden deutschen Text ins Englische zu übersetzen: \"{text}\" Lies dir die Anweisung jetzt nochmal durch: Stell dir vor, du bist ein professioneller Übersetzer mit jahrelanger Erfahrung. Deine Spezialität ist es, Texte mithilfe von direkten Übersetzungen präzise und unter Beibehaltung des ursprünglichen Tons von einer Sprache in eine andere zu übertragen. Deine Aufgabe ist es, ohne Rückfragen oder weiterführende Erklärungen den deutschen Text ins Englische zu übersetzen.',
        'prompt_language': Language.GERMAN,
        'prompt_type': 'zero_shot_style_persona_reread',
        'complexities': ALL_COMPLEXITIES
    },
    # ...
}

***
## 4 Execute Pipeline

In [14]:
translation_results = run_pipeline(data)
print('Fertig mit der Pipeline. Speichere Ergebnisse...')
translation_results.to_csv('translation_results.csv', sep=';')
print('Pipeline abgeschlossen. Ergebnisse gespeichert.')

llama_init_from_model: n_ctx_per_seq (512) < n_ctx_train (8192) -- the full capacity of the model will not be utilized


Model lmstudio-ai/gemma-2b-it-GGUF erfolgreich geladen mit n_ctx=512
estimated_max_tokens: 541.5; n_ctx: 595


llama_init_from_model: n_ctx_per_seq (608) < n_ctx_train (8192) -- the full capacity of the model will not be utilized


Model lmstudio-ai/gemma-2b-it-GGUF erfolgreich geladen mit n_ctx=595
Prompt finished in (seconds):  3.08
Metric Calculation in (seconds):  29.95
🏃 View run gemma/easy/zero_shot_style_persona_reread_to-en_en_1 at: http://127.0.0.1:5000/#/experiments/429765178055128713/runs/2f06c554d413479dae465fc6937f98b7
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/429765178055128713
Prompt finished in (seconds):  3.25
Metric Calculation in (seconds):  25.56
🏃 View run gemma/easy/zero_shot_style_persona_reread_to-en_de_1 at: http://127.0.0.1:5000/#/experiments/429765178055128713/runs/6f88c819f5484b3bbab586001c07cd6e
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/429765178055128713
Prompt finished in (seconds):  2.58
Metric Calculation in (seconds):  23.91
🏃 View run gemma/easy/zero_shot_style_persona_reread_to-de_en_1 at: http://127.0.0.1:5000/#/experiments/429765178055128713/runs/b499c794a87a4265aac5d0449f70352c
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/42976517

llama_init_from_model: n_ctx_per_seq (512) < n_ctx_train (8192) -- the full capacity of the model will not be utilized


...

Model lmstudio-community/Meta-Llama-3.1-8B-Instruct-GGUF erfolgreich geladen mit n_ctx=777
Prompt finished in (seconds):  116.11
Metric Calculation in (seconds):  65.9
🏃 View run llama31/pop_science/zero_shot_style_persona_reread_to-en_en_1 at: http://127.0.0.1:5000/#/experiments/358136329887025901/runs/35f7a6574e504fdfa45dbbe1c8ba4da1
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/358136329887025901
Prompt finished in (seconds):  102.1
Metric Calculation in (seconds):  50.43
🏃 View run llama31/pop_science/zero_shot_style_persona_reread_to-en_de_1 at: http://127.0.0.1:5000/#/experiments/358136329887025901/runs/610376c83ca14f548573abf07bb19ebc
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/358136329887025901
Prompt finished in (seconds):  112.09
Metric Calculation in (seconds):  57.04
🏃 View run llama31/pop_science/zero_shot_style_persona_reread_to-de_en_1 at: http://127.0.0.1:5000/#/experiments/358136329887025901/runs/9f6d2bbdfc7b47289507da69b93214ab
🧪 View experi

llama_init_from_model: n_ctx_per_seq (512) < n_ctx_train (131072) -- the full capacity of the model will not be utilized


Model lmstudio-community/Meta-Llama-3.1-8B-Instruct-GGUF erfolgreich geladen mit n_ctx=512
estimated_max_tokens: 1110.0; n_ctx: 1221


llama_init_from_model: n_ctx_per_seq (1248) < n_ctx_train (131072) -- the full capacity of the model will not be utilized


Model lmstudio-community/Meta-Llama-3.1-8B-Instruct-GGUF erfolgreich geladen mit n_ctx=1221
Prompt finished in (seconds):  179.87
Metric Calculation in (seconds):  145.8
🏃 View run llama31/science/zero_shot_style_persona_reread_to-en_en_1 at: http://127.0.0.1:5000/#/experiments/958464439758456234/runs/e5f97d97dcab4e8093d045c246b29a2d
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/958464439758456234
Prompt finished in (seconds):  164.97
Metric Calculation in (seconds):  117.2
🏃 View run llama31/science/zero_shot_style_persona_reread_to-en_de_1 at: http://127.0.0.1:5000/#/experiments/958464439758456234/runs/3545d71919be448cabeb89c4958490a7
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/958464439758456234
Prompt finished in (seconds):  195.77
Metric Calculation in (seconds):  162.26
🏃 View run llama31/science/zero_shot_style_persona_reread_to-de_en_1 at: http://127.0.0.1:5000/#/experiments/958464439758456234/runs/840be3f1a9da4eeda6a77b5bd96475df
🧪 View experiment at: